In [19]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_nomic import NomicEmbeddings

#from langchain_community.vectorstores import DocArrayHnswSearch
#from langchain_community.vectorstores import DocArrayInMemorySearch
#from langchain_community.retrievers import DocArrayRetriever

from langchain_core.vectorstores import InMemoryVectorStore

from langchain_text_splitters import RecursiveCharacterTextSplitter

from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [10]:
filepath = 'Datafiles/Publication606.pdf'
loader = PyMuPDFLoader(filepath)
docs = loader.load()
print(len(docs)) ## no of pages

4


In [11]:
print(f"{docs[0].page_content[:200]}\n") ## gives info of page 0
print(docs[0].metadata)

VOLUME 79, NUMBER 4
P H Y S I C A L R E V I E W L E T T E R S
28 JULY 1997
Propagation of Sound in a Bose-Einstein Condensate
M. R. Andrews, D. M. Kurn, H.-J. Miesner, D. S. Durfee, C. G. Townsend, S.

{'source': 'Datafiles/Publication606.pdf', 'file_path': 'Datafiles/Publication606.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Acrobat Distiller 2.1 for Power Macintosh', 'creationDate': 'D:19970715200452', 'modDate': "D:20001116073127-05'00'", 'trapped': ''}


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

25

In [14]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [15]:
vector_store = InMemoryVectorStore(embeddings)

In [16]:
ids = vector_store.add_documents(documents=all_splits)

In [18]:
results = vector_store.similarity_search_with_score("the nonlinear Schrödinger equation")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.5690146769640338

page_content='much smaller than the size of the condensate, we directly
observed the propagation of sound waves, analogous
to studies in superﬂuids, and determined the speed of
sound.
In this study, we demonstrated a method for
locally exciting a condensate using a focused off-resonant
laser beam, and a new technique of observing dynamical
processes using rapid sequencing of nondestructive phase-
contrast images.
Bose-Einstein condensed gases at temperature T ­ 0
are described by the nonlinear Schrödinger equation,
which, in the limit of a large number of atoms, yields hy-
drodynamic equations [1,9]. Density perturbations propa-
gate according to a wave equation for zeroth sound,
≠2
t dnsrd ­ = ? fc2srd=dnsrdg ,
(1)
where dnsrd is the deviation from the equilibrium density
nsrd.
The local speed of sound csrd is given by an
expression ﬁrst derived by Bogoliubov [10] and Lee,
Huang, and Yang [11]
csrd ­
q
nsrd ˜Uym ,
(2)
where ˜U ­ 4p ¯h2aym characterizes the (

In [21]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "Density perturbations propagate according to a wave equation for zeroth sound",
        "The density dependence of the speed of sound was studied using adiabatically expanded condensates.",
    ],
)

[[Document(id='5861625f-02bd-4550-afe8-e5c3c5249591', metadata={'source': 'Datafiles/Publication606.pdf', 'file_path': 'Datafiles/Publication606.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Acrobat Distiller 2.1 for Power Macintosh', 'creationDate': 'D:19970715200452', 'modDate': "D:20001116073127-05'00'", 'trapped': '', 'start_index': 1606}, page_content='gases, is the characterization of the system by its collective\nexcitations.\nSeveral experimental [6,7] and theoretical\n[8,9] papers have studied oscillations in Bose-Einstein\ncondensed gases of trapped atoms. So far, the resonance\nfrequencies and the damping of only the few lowest\nmodes have been investigated. Due to the small size of\nthe trapped clouds, the discrete nature of the spectrum\nis dominant, in contrast to the continuous spectrum of\nquantum liquids, which is phononlike at low frequencies.\nThe correspondence between th